In [15]:
%matplotlib inline
%config IPCompleter.greedy=True
import warnings
# Squash warning messages
warnings.showwarning = lambda *args, **kwargs: None

import pandas as pd

import sklearn
from sklearn import cluster
from sklearn import ensemble
from sklearn import feature_selection
from sklearn import metrics
from sklearn import model_selection
from sklearn import neural_network
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import r2_score
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

from ipywidgets import widgets
from ipywidgets import HBox, Label
from IPython.display import HTML

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import numpy as np

import pdb

import keras
from keras import models
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import itertools
from itertools import permutations

import IPython
# print(IPython.sys_info())


import math
import time
import seaborn as sns

import fancyimpute
from fancyimpute import KNN

from scipy import stats

print("Pandas ", pd.__version__)
print("Numpy ", np.__version__)

#Importing Pandas
Raw_Total_Data = pd.read_csv(r"C:\Users\ryan.hagan\Desktop\01_Preprocessed_WaterQuality_Data_2015_2019.csv", encoding = "ISO-8859-1")
                             
Raw_Target_Data = pd.read_csv(r"C:\Users\ryan.hagan\Desktop\01_Preprocessed_WaterQuality_Data_2015_2019.csv", encoding = "ISO-8859-1")

Pandas  0.25.1
Numpy  1.16.2


In [16]:
#Forcing data to be float
Raw_Total_Data.iloc[:, 1:] = Raw_Total_Data.iloc[:, 1:].apply(pd.to_numeric, downcast = 'float', errors = 'coerce')
Cleaned_Total_Data = Raw_Total_Data.iloc[:,1:].dropna()

Raw_Target_Data.iloc[:, 1:] = Raw_Target_Data.iloc[:, 1:].apply(pd.to_numeric, downcast = 'float', errors = 'coerce')
Cleaned_Target_Data = Raw_Target_Data.iloc[:,1:].dropna()

n_features = Cleaned_Target_Data.shape[1]
n_samples = Cleaned_Target_Data.shape[0]



print('Total number of features: ', str(n_features))
print('Total number of samples: ', str(n_samples))

Total number of features:  82
Total number of samples:  235


In [17]:
#First Algorithm to use is DBSCAN\n",
# #This is to check if the data geometry is uneven and clusters cannot be convex\n",
# #Applying to the whole system \n",
# DBSCAN_Output_System = sklearn.cluster.DBSCAN(metric='euclidean', algorithm='auto', n_jobs=-1).fit_predict(Cleaned_Total_Data)

# #Applying to one data stream at a time\n",
# DBSCAN_Output_Individual_Stream = Cleaned_Total_Data.copy()
# for Counter in range(0, Cleaned_Total_Data.shape[1]):
#     Input_Array = Cleaned_Total_Data.iloc[:,Counter].values
#     DBSCAN_Output_Individual_Stream.iloc[:, Counter] = sklearn.cluster.DBSCAN(metric='euclidean', algorithm='auto', n_jobs=-1).fit_predict(Input_Array.reshape(-1,1))

    
# DBSCAN_Silhoutte = metrics.silhouette_score(Cleaned_Total_Data, DBSCAN_Output_System)
# print(DBSCAN_Silhoutte)
# print('Results dimensions: ', str(DBSCAN_Output_System.ndim))
    

In [18]:
#Second algorithm to apply is Birch 
#This is a robust clustering algorithm used to detect outliers
Birch_Output_System = sklearn.cluster.Birch(n_clusters=2, compute_labels=True, copy=True).fit_predict(Cleaned_Total_Data)
Birch_Output_System3 = sklearn.cluster.Birch(n_clusters=3, compute_labels=True, copy=True).fit_predict(Cleaned_Total_Data)

#Applying to one data stream at a time
Birch_Output_Individual_Stream = Cleaned_Total_Data.copy()
Birch_Output_Individual_Stream3 = Cleaned_Total_Data.copy()

for Counter in range(0, Cleaned_Total_Data.shape[1]):
    Input_Array = Cleaned_Total_Data.iloc[:,Counter].values
    Birch_Output_Individual_Stream.iloc[:, Counter] = sklearn.cluster.Birch(n_clusters=2, compute_labels=True, copy=True).fit_predict(Input_Array.reshape(-1,1))
    Birch_Output_Individual_Stream3.iloc[:, Counter] = sklearn.cluster.Birch(n_clusters=3, compute_labels=True, copy=True).fit_predict(Input_Array.reshape(-1,1))

In [19]:
#Third algorithm is agglomorative clustering
#Applying to the whole system
AggCluster_Output_System = sklearn.cluster.AgglomerativeClustering(n_clusters=2, affinity='euclidean', compute_full_tree='auto', linkage='ward').fit_predict(Cleaned_Total_Data)

#Applying to one data stream at a time
AggCluster_Output_Individual_Stream = Cleaned_Total_Data.copy()
for Counter in range(0, Cleaned_Total_Data.shape[1]):
    Input_Array = Cleaned_Total_Data.iloc[:,Counter].values
    AggCluster_Output_Individual_Stream.iloc[:, Counter] = sklearn.cluster.AgglomerativeClustering(n_clusters=2, affinity='euclidean', compute_full_tree='auto', linkage='ward').fit_predict(Input_Array.reshape(-1,1))

In [20]:
#Applying the ensembling algoritm

#Analyzing overall system state
#First agglomerating features
Ensemble_System_Input = np.zeros((int(AggCluster_Output_System.shape[0]),3))
Ensemble_System_Input[:,0] = AggCluster_Output_System
Ensemble_System_Input[:,1] = Birch_Output_System
# Ensemble_System_Input[:,2] = DBSCAN_Output_System
Ensemble_System_TempStorage = sklearn.cluster.FeatureAgglomeration(n_clusters=2, affinity='euclidean', 
                                                              compute_full_tree='auto', linkage='ward').fit_transform(Ensemble_System_Input)
#Then applying clustering
Ensemble_Output_System = sklearn.cluster.AgglomerativeClustering(n_clusters=2, affinity='euclidean', compute_full_tree='auto', linkage='ward').fit_predict(Ensemble_System_TempStorage)

#Analyzing Individual Data Streams
Ensemble_Output_Individual_Stream = Cleaned_Total_Data.copy()
for Counter in range(0, Cleaned_Total_Data.shape[1]):
    Ensemble_Input_Individual_Stream = np.zeros((int(AggCluster_Output_System.shape[0]),3))
    Ensemble_Input_Individual_Stream[:,0] = AggCluster_Output_Individual_Stream.iloc[:,Counter].values
    Ensemble_Input_Individual_Stream[:,1] = Birch_Output_Individual_Stream.iloc[:,Counter].values
#     Ensemble_Input_Individual_Stream[:,2] = DBSCAN_Output_Individual_Stream.iloc[:,Counter].values
    
    Ensemble_Individual_Stream_TempStorage = sklearn.cluster.FeatureAgglomeration(n_clusters=2, affinity='euclidean', 
                                                                       compute_full_tree='auto', linkage='ward').fit_transform(Ensemble_Input_Individual_Stream)
    Ensemble_Output_Individual_Stream.iloc[:, Counter] = sklearn.cluster.AgglomerativeClustering(n_clusters=2, affinity='euclidean', compute_full_tree='auto', linkage='ward').fit_predict(Ensemble_Individual_Stream_TempStorage)


    

In [21]:
EnsembleModel_Silhoutte = metrics.silhouette_score(Cleaned_Total_Data, Ensemble_Output_System, metric='euclidean')
Aggcluster_Silhoutte = metrics.silhouette_score(Cleaned_Total_Data, AggCluster_Output_System, metric='euclidean')
Birch_Silhoutte = metrics.silhouette_score(Cleaned_Total_Data, Birch_Output_System, metric='euclidean')

#DBSCAN_Silhoutte = metrics.silhouette_score(Cleaned_Total_Data, DBSCAN_Output_System)

print('Ensemble Model Silhoutte Score is ' + str(EnsembleModel_Silhoutte) + ' AggCluster Silhoutte Score is ' + str(Aggcluster_Silhoutte) + ' Birch Silhoutte Score is ' + str(Birch_Silhoutte))

Ensemble Model Silhoutte Score is 0.37830228 AggCluster Silhoutte Score is 0.37830228 Birch Silhoutte Score is 0.37830228


In [22]:
EnsembleModel_Silhoutte = metrics.davies_bouldin_score(Cleaned_Total_Data, Ensemble_Output_System)
Aggcluster_Silhoutte = metrics.davies_bouldin_score(Cleaned_Total_Data, AggCluster_Output_System)
Birch_Silhoutte = metrics.davies_bouldin_score(Cleaned_Total_Data, Birch_Output_System)
#DBSCAN_Silhoutte = metrics.davies_bouldin_score(Cleaned_Total_Data, DBSCAN_Output_System)

# print('Ensemble Model Davies-Bouldin Index is ' + str(EnsembleModel_Silhoutte) + ' AggCluster Davies-Bouldin Index is ' + str(Aggcluster_Silhoutte) + ' Birch Davies-Bouldin Index is ' + str(Birch_Silhoutte))

In [24]:
#Saving the results of Anomaly Detection Algorithm

Ensemble_Output_System_Dataframe = pd.DataFrame(data= Ensemble_Output_System, index = Cleaned_Total_Data.index.values)
Birch_Output_System_Dataframe = pd.DataFrame(data= Birch_Output_System, index = Cleaned_Total_Data.index.values)
Ensemble_Output_Individual_Stream_Dataframe = pd.DataFrame(data= Ensemble_Output_Individual_Stream, index = Cleaned_Total_Data.index.values,
                                    columns = Cleaned_Total_Data.columns.values)


Birch_Output_Individual_Stream_Dataframe = pd.DataFrame(data= Birch_Output_Individual_Stream, index = Cleaned_Total_Data.index.values,
                                    columns = Cleaned_Total_Data.columns.values)


Agg_Output_System_Dataframe = pd.DataFrame(data= AggCluster_Output_System, index = Cleaned_Total_Data.index.values)

Agg_Output_Individual_Stream_Dataframe = pd.DataFrame(data= AggCluster_Output_Individual_Stream, index = Cleaned_Total_Data.index.values,
                                    columns = Cleaned_Total_Data.columns.values)



Ensemble_Output_System_Dataframe.to_csv(r"C:\Users\ryan.hagan\Desktop\01_WaterQuality_System_Status_2019.csv")
Birch_Output_System_Dataframe.to_csv(r"C:\Users\ryan.hagan\Desktop\01_WaterQuality_System_Status_All_BIRCH.csv")
Birch_Output_Individual_Stream_Dataframe.to_csv(r"C:\Users\ryan.hagan\Desktop\01_WaterQuality_Stream_Status_ALL_BIRCH_INDIVIDUAL.csv")
Ensemble_Output_Individual_Stream_Dataframe.to_csv(r"C:\Users\ryan.hagan\Desktop\01_WaterQuality_Stream_Status_2019.csv")

Agg_Output_System_Dataframe.to_csv(r"C:\Users\ryan.hagan\Desktop\01_WaterQuality_System_Status_All_Aggcluster.csv")
Agg_Output_Individual_Stream_Dataframe.to_csv(r"C:\Users\ryan.hagan\Desktop\01_WaterQuality_Stream_Status_ALL_Aggcluster_INDIVIDUAL.csv")
